In [ ]:
# nse scratchpad initiation
from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

ib =  get_connected('nse', 'live')

with open(logpath+'spad.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'spad.log')

In [ ]:
# nse_main.py
# Main program for nse

from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
    
ip = 1

# do the appropriate function
if ip in [1, 2]:    #  build ALL the options
    
    with get_connected('nse', 'live') as ib:
        
        with open(logpath+'build.log', 'w'):
            pass # clear the run log
        
        util.logToFile(logpath+'build.log')
        
        util.logging.info('####                     NSE BUILD STARTED                  ####')
        print(f'NSE options build started at {datetime.datetime.now()}...')
        s = time.perf_counter()
        
        # generate the symexplots
        # df_l = symexplots(ib)
        df_l = pd.read_pickle(fspath+"symexplots.pickle")
        
        # get the series of rows for options
        rows = [s for i, s in df_l.iterrows()]
        
        if ip is 2:
            # remove rows already pickled
            pkl_done = pd.read_pickle(fspath+'opts.pickle').symbol.unique()
            rows = [row for row in rows if row.symbol not in pkl_done]

        # get the options
        from tqdm import trange
        tqr = trange(len(rows), desc='Processing', leave=True) # Initializing tqdm
        opts = [] # initializing list of opts
        
        async def opt_gen(ib, row, df_l):
            return await asyncio.gather(do_an_opt(ib, row, df_l))

        for row in rows:
            tqr.set_description(f"Processing [{row.symbol}]")
            tqr.refresh() # to show immediately the update    
            #loop = asyncio.get_event_loop()
            #loop.run_until_complete(opt_gen(ib, row, df_l))
            opt_gen(ib, row, df_l)
        tqr.close()